In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
#create the SparkSession
spark = SparkSession.\
    builder.\
    appName("sparkStreaming").\
    getOrCreate()

In [4]:
# Define input sources
lines = (spark\
.readStream.format("socket")\
.option("host", "localhost")\
.option("port", 9999)\
.load())

In [5]:
# Transform data
words = lines.select(split(col("value"), "\\s").alias("word"))

In [6]:
# Get the count of published words
counts = words.groupBy("word").count()

In [8]:
# Define the checkpoint directory
checkpointDir = "D:/code/python/spark/checkpoint/"

In [9]:
# Start streaming defining the necessary configurations
streamingQuery = (counts
.writeStream
.format("console")
.outputMode("complete")
.trigger(processingTime="1 second")
.option("checkpointLocation", checkpointDir)
.start())
streamingQuery.awaitTermination()